In [ ]:
import numpy as np
import seaborn as sns
import scipy.stats as spy
from numba import njit, prange

sns.set()      
sns.set_context("talk")

def fact_arr(n):
    # takes the factorial of an array "n" of values
    fact_n=np.zeros(n.shape[0])
    for i in range(n.shape[0]):
        fact_n[i]=np.math.factorial(n[i])
    return fact_n
def p_Poiss(mu,n):
    # returns the probability of n events for a mean value of mu in a Poisson process
    return mu**n*np.exp(-mu)/fact_arr(n)
def port_prob(mu,d_arr,sort,eta):
    # using the system properties (mean photon number "mu", dark counts array "d_arr", polarization sorting matrix "sort",
    # channel loss "eta"), generates a lookup table of probabilties of outputs (columns) for different inputs (rows).  
    # 5th row is vacuum state.
    n_pol_states = sort.shape[0]
    n_detectors = len(d_arr)
    p_click_mat = np.zeros([n_pol_states+1,n_detectors])
    mu_mat = sort@eta*mu
    for polar in range(n_pol_states):
        for port in range(n_detectors):
            p_click_mat[polar,port] = 1-(p_Poiss(mu_mat[polar,port],np.array([0.]))*(1-d_arr[port]))
    p_click_mat[n_pol_states,:] = np.transpose(d_arr)
    return p_click_mat

def nlik_prob(p_click_mat,p_send_arr):
    # returns the weighted average detection probability for the different detectors using the input-output lookup
    # table "p_click_mat" and the frequencies of the different inputs "p_send_arr"
    n_detectors = p_click_mat.shape[1]
    #nlik_arr = 1-np.prod(1-p_click_mat,0)
    nlik_arr=np.ones(n_detectors)
    for port in range(n_detectors):
        for polar in range(len(p_send_arr)):
            nlik_arr[port] = nlik_arr[port]*(1-p_send_arr[polar]*p_click_mat[polar,port])
            # first finds the approximate probability of getting no clicks
    return 1-nlik_arr
def DataSifter(Test,Stored,d=2):
    """Takes Alice's (Test) and Bob's (Stored) synchronized data strings, and sifts them.  Yields Alice and Bob's nearly 
    identical sifted arrays (dark counts prevent this from being perfect), and the boolean mask that produced them."""
    # d is the dimension of the basis, but implementation of d dimensional synchronization is incomplete
    Basis_A=np.array([np.sum(Test[:d,:],0),np.sum(Test[d:,:],0)])
    # 1 is H/V, 2 is R/L
    Basis_B=np.array([np.sum(Stored[:d,:],0),np.sum(Stored[d:,:],0)])
    #overlap=(1-Stored[0,:]*Stored[1,:])*(1-Stored[2,:]*Stored[3,:])*(1-Basis_B[0,:]*Basis_B[1,:])
    overlap = (np.sum(Stored,0)==1)*1  # exactly one detection
    Basis_B[0,:]=Basis_B[0,:]*overlap
    Basis_B[1,:]=Basis_B[1,:]*overlap
    sift=sum(Basis_A*Basis_B)
    sifter=np.zeros(Test.shape)
    for i in range(d*2):
        sifter[i,:]=sift
    SiftedTest=np.reshape(Test[sifter==np.ones(sifter.shape)],[d*2,int(sum(sift))])
    SiftedStored=np.reshape(Stored[sifter==np.ones(sifter.shape)],[d*2,int(sum(sift))])
    return SiftedTest, SiftedStored, sift
def Syncing_metric(Test,Stored,not_Stored,N_prior,p_click_mat,nlik_arr):
    """"Computes the synchronization metric for the Syncing_try function"""
    #Test5 = np.zeros([5,Test.shape[1]])
    #Test5[0:4,:] = Test
    #Test5[4,:] = (np.sum(Test,0)==0)*1
    count_mat = Test@(np.transpose(Stored))
    no_count_mat = Test@(np.transpose(not_Stored))
    
    lik = np.log(1/N_prior)
    nlik=np.log((N_prior-1)/N_prior)
    for i in range(Test.shape[0]):
        for j in range(Stored.shape[0]):
            lik = lik+np.log(p_click_mat[i,j])*count_mat[i,j]+np.log(1-p_click_mat[i,j])*no_count_mat[i,j]
            nlik = nlik+np.log(nlik_arr[j])*count_mat[i,j]+np.log(1-nlik_arr[j])*no_count_mat[i,j]
    
    if ((lik-nlik)<700) & ((lik-nlik)>-700):
        return np.exp((lik-nlik))/(np.exp(lik-nlik)+np.exp(1))
    elif (lik-nlik)>700:
        return 1
    else:
        return 0

def Syncing_try(TestPatch,StoredPatch,sRange,recovered_length,mu_sig,mu_dec,d_arr,sort,eta,p_send_arr,Ntries=0,Mask=[-1]):
    """Simplified, nearly deprecated, slow, and mathematically tenuous way to the synchronization probability.  Uses a for 
    loop to try different synchronization points.  Consider using Syncing_fft."""
    n_pol_states = sort.shape[0]
    n_detectors = len(d_arr)
    not_StoredPatch = 1 - StoredPatch
    if Mask[0] == -1:
        Mask=np.ones([TestPatch.shape[1]])
    FullMask = np.zeros(TestPatch.shape)
    for i in range(TestPatch.shape[0]):
        FullMask[i,:] = Mask  
        # one might want to apply a boolean mask to choose which points to use for synchronization
        # but I longer find this a useful feature since I no longer burn qubits for synchronization
    TestPatch = TestPatch * FullMask
    N=Ntries # number of points to try, basically deprecated
    if Ntries==0:
        N=StoredPatch.shape[1]-TestPatch.shape[1]+1
    metric=np.zeros(recovered_length)
    p_click_mat_sig = port_prob(mu_sig,d_arr,sort,eta)
    p_click_mat_dec = port_prob(mu_dec,d_arr,sort,eta)
    # full 9 input state lookup table (4 signals, 4 decoys, 1 vacuum)
    p_click_mat = np.zeros([TestPatch.shape[0],StoredPatch.shape[0]]) 
    p_click_mat[:n_pol_states,:] = p_click_mat_sig[:n_pol_states,:]
    p_click_mat[n_pol_states:,:] = p_click_mat_dec
    nlik_arr = nlik_prob(p_click_mat,p_send_arr)
    nlik_mat = np.zeros(p_click_mat.shape)
    for i in range(nlik_mat.shape[0]):
        nlik_mat[i,:] = nlik_arr
        
    for i in range(N):
        StoredPatch_short=StoredPatch[:,i:i+TestPatch.shape[1]]
        not_StoredPatch_short=not_StoredPatch[:,i:i+TestPatch.shape[1]]
        metric[i+sRange[0]]=Syncing_metric(TestPatch,StoredPatch_short,not_StoredPatch_short,N,p_click_mat,nlik_arr)
    return metric


def Syncing_fft(TestPatch,StoredPatch,sRange,recovered_length,\
                mu_sig,mu_dec,d_arr,sort,eta,size,num_blocks,p_send_arr,Ntries=0,Mask=[-1],normalize=1,TestStart=0):
    """Uses a a fast fourier transform to count the numbers of different events along with a probability lookup table to
    compute the synchronization probability using TestPatch at different time offsets of Stored.  Normalize = 1 is 
    preferred (and mathematically justifiable) method."""
    n_pol_states = sort.shape[0]
    n_detectors = len(d_arr)
    if Mask[0] == -1:
        Mask=np.ones(TestPatch.shape[1])
    TestMask = np.zeros(TestPatch.shape)
    #StoredPatch = Stored[:,sRange[0]:(sRange[1]+Ntries)]
    for i in range(TestPatch.shape[0]):
        TestMask[i,:] = Mask
        # one might want to apply a boolean mask to choose which points to use for synchronization
        # but I longer find this a useful feature since I no longer burn qubits for synchronization
    TestPatch = TestPatch*TestMask
    #BlankPatch = 1-np.sum(TestPatch,0)
    padded_length = next_power(StoredPatch.shape[1],2) # ffts want things padded with zeros to the next power of 2
    BlankPatch = np.zeros([2,padded_length])
    #BlankPatch[0,:] = np.ones(padded_length)
    for i in range(padded_length): # making note of when something was sent and when nothing was sent across whole data
        BlankPatch[0,i] = ((((i+TestStart)%size)>=0) & (((i+TestStart)%size)<num_blocks))
        BlankPatch[1,i] = (((i+TestStart)%size)>=num_blocks)
    TestPatch_padded = np.zeros([TestPatch.shape[0],padded_length])
    TestPatch_padded[:,:TestPatch.shape[1]] = TestPatch
    StoredPatch_padded = np.zeros([StoredPatch.shape[0],padded_length])
    StoredPatch_padded[:,:StoredPatch.shape[1]] = StoredPatch
    NotStoredPatch_padded = np.zeros([StoredPatch.shape[0],padded_length])
    NotStoredPatch_padded[:,:StoredPatch.shape[1]] = (1-StoredPatch)
    #BlankPatch_padded = np.zeros([padded_length])
    #BlankPatch_padded[:BlankPatch.shape[0]] = BlankPatch
    
    if Ntries==0: # number of points to try, basically deprecated
        Ntries=StoredPatch.shape[1]-TestPatch.shape[1]+1
    N_prior = Ntries # basic prior is one over the number of candidates
    #print(N_prior)
    #N_prior=2
    metric=np.zeros(recovered_length-TestPatch.shape[1]+1)
    p_click_mat_sig = port_prob(mu_sig,d_arr,sort,eta)
    p_click_mat_dec = port_prob(mu_dec,d_arr,sort,eta)
    p_click_mat = np.zeros([TestPatch.shape[0],StoredPatch.shape[0]])
    # full 9 input state lookup table (4 signals, 4 decoys, 1 vacuum)
    p_click_mat[:n_pol_states,:] = p_click_mat_sig[:n_pol_states,:]
    p_click_mat[n_pol_states:,:] = p_click_mat_dec
    nlik_arr = nlik_prob(p_click_mat,p_send_arr) # uninformed detection probabilities
    nlik_mat = np.zeros(p_click_mat.shape) # nice to have it in a matrix form
    for i in range(nlik_mat.shape[0]):
        nlik_mat[i,:] = nlik_arr
    # counting up the different event combinations using ffts
    count_mat = np.zeros([padded_length,TestPatch.shape[0],StoredPatch.shape[0]])
    no_count_mat = np.zeros([padded_length,TestPatch.shape[0],StoredPatch.shape[0]])
    blank_mat = np.zeros([padded_length,2,StoredPatch.shape[0]])
    for j in range(StoredPatch.shape[0]):
        for i in range(TestPatch.shape[0]):
            count_mat[:,i,j] = np.round(np.real(np.fft.irfft(np.conj(np.fft.rfft(TestPatch_padded[i,:]))\
                                                           *np.fft.rfft(StoredPatch_padded[j,:]))))
            #no_count_mat[:,i,j] = np.round(np.real(np.fft.irfft(np.conj(np.fft.rfft(TestPatch_padded[i,:]))\
            #                                               *np.fft.rfft(NotStoredPatch_padded[j,:]))))
        blank_mat[:,0,j] = np.round(np.real(np.fft.irfft(np.conj(np.fft.rfft(BlankPatch[0,:]))\
                                                           *np.fft.rfft(StoredPatch_padded[j,:]))))
        blank_mat[:,1,j] = np.round(np.real(np.fft.irfft(np.conj(np.fft.rfft(BlankPatch[1,:]))\
                                                           *np.fft.rfft(StoredPatch_padded[j,:]))))
    no_count_mat = np.zeros(count_mat.shape)
    no_blank_mat = np.zeros(blank_mat.shape)
    num_sent = np.sum(TestPatch_padded,1)
    blank_sent = np.sum(BlankPatch[:,:StoredPatch.shape[1]],1)
    for g in range(count_mat.shape[2]):
        no_count_mat[:,:,g] = num_sent
        no_blank_mat[:,:,g] = blank_sent
    no_count_mat = no_count_mat-count_mat
    no_blank_mat = no_blank_mat-blank_mat
    #print(blank_mat[:30,:,:])
    if normalize==0: # not assuming a unique sync point, but the math may also be fishy
        for k in range(Ntries):
            lik = 0#np.log(1/N_prior)
            nlik=np.log((N_prior-1)/N_prior)
            lik = lik+np.sum(np.sum(np.log(p_click_mat)*count_mat[k,:,:]\
                +np.log(1-p_click_mat)*no_count_mat[k,:,:]))
            nlik = nlik+np.sum(np.sum(np.log(nlik_mat)*count_mat[k,:,:]\
                +np.log(1-nlik_mat)*no_count_mat[k,:,:]))
            
            if ((lik-nlik)<700) & ((lik-nlik)>-700):
                metric[sRange[0]+k] = np.exp((lik-nlik))/(np.exp(lik-nlik)+1)
            elif (lik-nlik)>700:
                metric[sRange[0]+k] = 1
            else:
                metric[sRange[0]+k] = 0
    whichpoint = 1
    if normalize==1: # assuming unique sync point, math is good
        lik = np.zeros(Ntries)
        nlik = np.zeros(Ntries)
        unlik = np.zeros(Ntries)
        quo = np.zeros(Ntries)
        for k in range(Ntries):
            # these are *basically* the 3 terms in the expression at the end of the theory section
            lik[k] = np.sum(np.sum(np.log(p_click_mat)*count_mat[k,:,:]\
                +np.log(1-p_click_mat)*no_count_mat[k,:,:]))
            nlik[k] = np.sum(np.sum(np.log(nlik_mat)*count_mat[k,:,:]\
                +np.log(1-nlik_mat)*no_count_mat[k,:,:]))
            #print(lik[k])#,nlik[k])
            unlik[k] = np.sum(np.log(nlik_mat)*blank_mat[k,0,:]+np.log(1-nlik_mat)*no_blank_mat[k,0,:]\
                +np.log(p_click_mat[-1,:])*blank_mat[k,1,:]+np.log(1-p_click_mat[-1,:])*no_blank_mat[k,1,:])
        # play some programming games to avoid underflow/overflow issues
        quodif = lik -nlik +unlik
        ind = np.where(quodif == np.max(quodif))[0][0]
        if ind==0:
            whichpoint = 1
        elif ind==(Ntries-1):
            whichpoint = 0
        else:
            whichpoint=(quodif[ind-1]<quodif[ind+1]) # which neighboring bin is being straddled, we want that too
        for k in range(Ntries):
            if (quodif[k]-quodif[ind])<-700:
                quo[k] = 0
            else:
                quo[k] = np.exp(quodif[k]-quodif[ind])
        metric[sRange[0]:sRange[0]+Ntries] = quo/np.sum(quo)
        #metric_data_map = np.memmap('metric_data_mem.npy', dtype='float64',mode = 'w+', shape=(Ntries))
        #metric_data_map[:] = metric[sRange[0]:sRange[0]+Ntries].T
        #del metric_data_map
        #for k in range(Ntries):
            #print(np.log(metric[sRange[0]+k]))
        #print(np.sum(metric))
        print(np.sum(quo))
    return metric, whichpoint

def next_power(size_old,base): # find the next power of 2, useful for padding arrays before the ffts
    size_new = 1
    while size_old>size_new:
        size_new *= base
    return size_new
def sync_pt(TestStart,TestPatch,StoredPatch,sRange,recovered_length,\
            mu_sig,mu_dec,d_arr,sort,eta,size,num_blocks,p_send_arr,method='fft',Ntries=0,Mask=[-1],normalize=0):
    # this function is mostly error handing and displaying results from the syncing methods
    if method == 'fft':
        metric, whichpoint = Syncing_fft(TestPatch,StoredPatch,sRange,recovered_length,\
                             mu_sig,mu_dec,d_arr,sort,eta,size,num_blocks,p_send_arr,Ntries,Mask,normalize,TestStart)
    if method == 'shift':
        whichpoint = 1
        metric = Syncing_try(TestPatch,StoredPatch,sRange,recovered_length,\
                             mu_sig,mu_dec,d_arr,sort,eta,p_send_arr,Ntries=0,Mask=[-1])
    sync_arr = np.where(metric==np.max(metric))[0]
    sync_point = sync_arr[int(np.floor(np.random.uniform(0,1)*sync_arr.shape[0]))]-TestStart
    num_sync = sync_arr.shape[0]
    #num_sync = np.sum(metric>.5)
    two_points = 0
    #print(metric)
    if num_sync>1:
        print('Sync point not unique:')
    if num_sync==2:
        print('2 viable points')
        two_points = 1
        #print(np.where(metric==np.max(metric))[0][1]-TestStart)
    if num_sync>2:
        print('multiple viable points')
        print(num_sync)
    if np.max(metric)<.5:
        print('Correlation is weaker than .5')
    return sync_point, np.max(metric), whichpoint
def make_bin_array(arr,nbits): # turns a 1d array into nbits rows 2d matrix where the rows are the binary digits
    new_arr=np.zeros([nbits,len(arr)])
    for j in range(nbits):
        new_arr[j,:]=(arr//(2**j))%2
    final_arr=np.zeros([nbits+4,len(arr)])
    final_arr[4:,:]=new_arr
    return final_arr
def port_prob_parallel(mu,d_arr,sort,eta): 
    # wanted to parallelize the port_prob function for use with many different mpns
    n_hits = len(mu)
    #print(n_hits)
    n_pol_states = sort.shape[0]
    n_detectors = len(d_arr)
    p_click_mat = np.zeros([n_pol_states+1,n_detectors,n_hits])
    split = (sort@eta)
    mu_mat = np.zeros([n_pol_states,n_detectors,n_hits])
    d_mat = np.zeros([n_pol_states,n_detectors,n_hits])
    for polar in range(n_pol_states):
        for port in range(n_detectors):
            mu_mat[polar,port,:] = mu*split[polar,port]
            d_mat[polar,port,:] = d_arr[port]*np.ones(n_hits)
    p_click_mat[:n_pol_states,:,:] = 1-((spy.poisson.pmf(0,mu_mat))*(1-d_mat))
    p_click_mat[n_pol_states,:,:] = d_mat[0,:,:]
    return p_click_mat
def FakeData_R(Test,mpn_arr,mu_sig,mu_dec,d_arr,sort,eta):
    # parallelized measurement simulation for making realistic fake data
    n_pol_states = sort.shape[0]
    n_detectors = len(d_arr)
    n_hits = len(mpn_arr)
    p_click_mat = np.zeros([Test.shape[0],n_detectors,n_hits])
    Test_arr = np.zeros([Test.shape[0],n_detectors,n_hits])
    #print(2)
    for j in range(n_detectors):
        Test_arr[:,j,:] = Test
    #print(3)
    p_click_mat_sig = port_prob_parallel(mu_sig*mpn_arr,d_arr,sort,eta)
    p_click_mat_dec = port_prob_parallel(mu_dec*mpn_arr,d_arr,sort,eta)
    p_click_mat[:n_pol_states,:,:] = p_click_mat_sig[:-1,:]
    p_click_mat[n_pol_states:,:,:] = p_click_mat_dec
    #print(4)
    rand=np.random.uniform(0,1,[n_detectors,Test.shape[1]])
    Stored = (np.sum(Test_arr*p_click_mat,0)>rand)
    #Stored = ((np.transpose(p_click_mat)@Test)>rand)
    #print(5)
    return Stored*1
def Condense_data(FullData,det):
    # For faking data storage
    # In the experiment we only record timetags of detections.  This throws out the non-detection events from the fake 
    # data, so that we can show we can recover them again, just as we would for real data
    nonempty = np.zeros(FullData.shape[1])
    for i in range(len(det)):
        nonempty=nonempty+np.array([FullData[det[i],:]])  # changed to isolate H detections
    CondMask=(nonempty>0)
    FullMask=np.zeros(FullData.shape)
    for i in range(FullData.shape[0]):
        FullMask[i,:]=CondMask
    CondData=np.reshape(FullData[FullMask==np.ones(FullMask.shape)],[FullData.shape[0],int(sum(sum(CondMask)))])
    return CondData
def read_time_tag(TimeData):
    # turns my N x nbits time array back from binary to decimal
    Time_dec=np.zeros(int(TimeData.size/TimeData.shape[0]))
    if len(Time_dec)>1:
        for i in range(TimeData.shape[0]):
            Time_dec=Time_dec+TimeData[i,:]*2**i
    else:
        for i in range(TimeData.shape[0]):
            Time_dec=Time_dec+TimeData[i]*2**i
    return Time_dec
def ExpandData_sep(TimeData,det):  # det is which detector
    # Looking at the gaps between the recorded time tags, fill in all the implied no-detection events
    TimeDiff=np.diff(TimeData)
    Gaps=np.zeros([1,TimeDiff.shape[1]])
    for i in range(TimeData.shape[0]):
        Gaps=Gaps+TimeDiff[i,:]*2**i
    del TimeDiff
    Gaps=np.array(np.mod(Gaps,int(2**TimeData.shape[0])))
    ExpData_r=np.zeros(int(np.sum(Gaps[0]))+1)
    for i in range(TimeData.shape[1]-1):
        ExpData_r[int(np.sum(Gaps[0][:(i+1)])-Gaps[0][i])]=1
    ExpData_r[int(np.sum(Gaps))]=1
    return ExpData_r
def Make_Gaps(TimeData_H):
    # take an N long array of time tags and makes an N-1 long array of gaps
    # used with gap_prob to identify signal regions of the data
    rollover=2**TimeData_H.shape[0]
    TimeDiff=np.diff(TimeData_H)
    Gaps=np.zeros([1,TimeDiff.shape[1]])
    for i in range(TimeData_H.shape[0]):
        Gaps=Gaps+TimeDiff[i,:]*2**i
    Gaps=np.array(np.mod(Gaps,rollover))
    Gaps=Gaps[0]
    return Gaps
def Join_columns(*args): # this seems to join rows...
    # after all the signal regions are recovered for the 4 channels, this slaps them together into one rectified array
    #lengths=args.size
    args=args[0]
    lengths=np.zeros([len(args)])
    for i in range(len(args)):
        lengths[i]=len(args[i])
    #print(max(lengths))
    joined=np.zeros([len(args),int(max(lengths))])
    for i in range(len(args)):
        joined[i,:int(lengths[i])]=args[i]
    return joined
def gap_prob(gap,prob,rollover,eps):
    # naive gap probabilities
    prob_sum=prob*(1-prob)**(gap-1)
    while ((prob_sum-(prob*(1-prob)**(gap-1+rollover)))/prob_sum)<(1-eps):
        gap=gap+rollover
        prob_sum=prob_sum+prob*(1-prob)**(gap-1)
    return prob_sum
def gap_prob_sig(gap,p_s,p_d,size,num_blocks):
    # gap probabilities considering duty cycle and frequency of signals
    sig_term = 0
    if ((gap-1)%size)<(num_blocks-1):
        aold = (gap%size)-1
    else:
        aold = num_blocks-1 
    bold = ((gap-1)%size)-aold
    for i in range(num_blocks):
        if ((i+gap)%size)<num_blocks:
            sig_end = 1
        else:
            sig_end = 0
        sig_term += (1-(p_s+p_d))**aold * (1-p_d)**bold * (p_s+p_d)**sig_end * p_d**(1-sig_end)
        if ((gap+i)%size)>=num_blocks:
            aold -= 1
            bold += 1
    dark_term = 0
    if ((gap-1)%size)<(size-num_blocks-1):
        bold = (gap%size)-1
    else:
        bold = size-num_blocks-1 
    aold = ((gap-1)%size)-bold 
    for i in range(size-num_blocks):
        if ((i+gap)%size)<(size-num_blocks):
            dark_end = 1
        else:
            dark_end = 0
        dark_term += (1-(p_s+p_d))**aold * (1-p_d)**bold * p_d**dark_end * (p_s+p_d)**(1-dark_end)
        if ((gap+i)%size)>=(size-num_blocks):
            aold += 1
            bold -= 1
    prob_sum = ((1-(p_s+p_d))**num_blocks * (1-p_d)**(size-num_blocks))**(divmod(gap-1,size)[0]) *\
    (((p_s+p_d)*sig_term + p_d*dark_term)/(num_blocks*(p_s+p_d)+(size-num_blocks)*p_d))
    return prob_sum
def gap_prob_rollover(gap,p_s,p_d,size,num_blocks,rollover,eps):
    # adding probability contributions from number equal to the gap modulo rollover
    # keep adding them until they are extremely irrelevant
    prob_sum_old = gap_prob_sig(gap,p_s,p_d,size,num_blocks)
    gap += rollover
    prob_sum_new = gap_prob_sig(gap,p_s,p_d,size,num_blocks)
    while (prob_sum_new/prob_sum_old)>eps:
        prob_sum_old += prob_sum_new
        gap=gap+rollover
        prob_sum_new = gap_prob_sig(gap,p_s,p_d,size,num_blocks)
    return prob_sum_old
def likelihood_arrays(gaps,p_poor,p_rich,size,num_blocks,rollover,eps):
    # make arrays of the probabilities of each gap being a member of signal or dark regions
    # this will save us time later and all we will need to do is index these arrays
    g1=np.zeros(len(gaps))
    g2=np.zeros(len(gaps))
    for i in range(len(gaps)):
        g1[i]=np.log(gap_prob(gaps[i],p_poor,rollover,eps))
        g2[i]=np.log(gap_prob_rollover(gaps[i],p_rich,p_poor,size,num_blocks,rollover,eps))
    return g1,g2
def log_likelihood(theta,g1,g2,gaps,p_poor,p_rich,rollover,eps):
    # computes a likelihood of the gaps given transition points 
    if 0<(theta[0])<len(gaps) and 0<(theta[1])<len(gaps):# and (theta[0])<(theta[1]):    
        theta1=int(theta[0])
        theta2=int(theta[1])
        dark1=np.sum(g1[:theta1])
        sig=np.sum(g2[theta1:theta2])
        dark2=np.sum(g1[theta2:])
        return dark1+sig+dark2
    else:
        return 1
def log_prior_gauss(theta,gaps,ndim): # not gauss btw
    # uniform prior for the position of the transition points, also enforces the order of the transition points
    term=np.zeros(ndim)
    #print(theta)
    if (theta[0])<(theta[1]):
        term_other=0
    else:
        term_other=-np.inf
    for i in range(ndim):
        if 0<theta[i]<len(gaps):
            term[i] =  0
        else:
            term[i] = -np.inf
    return np.sum(term)+term_other
    #return -theta**2 / 50
def log_posterior_gauss(theta,g1,g2,gaps,p_poor,p_rich,rollover,eps,ndim):  # I lied about the gauss
    # likelihood plus prior equals posterior
    return log_prior_gauss(theta,gaps,ndim) + log_likelihood(theta,g1,g2,gaps,p_poor,p_rich,rollover,eps)
def search_bins(emcee_trace,Nbins):
    # after making the mcmc trace, want to efficiently find the optimal points
    a1_new=0
    a2_new=0
    b1_new=Nbins
    b2_new=Nbins
    a1_old=0
    a2_old=0
    b1_old=Nbins
    b2_old=Nbins
    
    while (b1_old-a1_old) > 2:
        a1_old=a1_new
        a2_old=a2_new
        b1_old=b1_new
        b2_old=b2_new
        Nbins_rt=int(np.ceil(np.sqrt(Nbins)))
        if (b1_old-a1_old) < 100:
            Nbins_rt=int(np.ceil(b1_old)-np.floor(a1_old)+1)
            a1_old=np.floor(a1_old)
            a2_old=np.floor(a2_old)
            b1_old=np.ceil(b1_old)
            b2_old=np.ceil(b2_old)
        #print('Nbins_rt = ',Nbins_rt)
        #print(type(Nbins_rt+1))
        #print(Nbins_rt+1)
        mc_hist1=plt.hist(emcee_trace[0],bins=np.linspace(a1_old,b1_old,Nbins_rt+1))
        mc_hist2=plt.hist(emcee_trace[1],bins=np.linspace(a2_old,b2_old,Nbins_rt+1))
        #print(mc_hist2)
        ind1=np.where(mc_hist1[0]==max(mc_hist1[0]))[0][0]
        ind2=np.where(mc_hist2[0]==max(mc_hist2[0]))[0][0]
        #print('ind2 = ',ind2)
        a1_new=ind1*(b1_old-a1_old)/Nbins_rt+a1_old
        a2_new=ind2*(b2_old-a2_old)/Nbins_rt+a2_old
        b1_new=(ind1+1)*(b1_old-a1_old)/Nbins_rt+a1_old
        b2_new=(ind2+1)*(b2_old-a2_old)/Nbins_rt+a2_old
        Nbins=Nbins_rt
    transition_pt=np.zeros(2)
    transition_pt[0]=int(b1_old)+1
    transition_pt[1]=int(a2_old)-1
    #print(transition_pt)
    return transition_pt

import emcee
print('emcee sampling (version: )', emcee.__version__)
import matplotlib.pyplot as plt

def find_transition(Gaps,p_poor,p_rich,size,num_blocks,rollover,eps):
    # use an mcmc to explore and optimize the two parameters (the two transition points)
    ndim = 2  # number of parameters in the model
    nwalkers = 50  # number of MCMC walkers
    nsteps = 2000 # steps per walker
    nburn = 1000
    
    g1,g2=likelihood_arrays(Gaps,p_poor,p_rich,size,num_blocks,rollover,eps)
    print(f'{nwalkers} walkers: {nsteps} samples each')
    # initialize walkers
    starting_guesses = np.random.randn(nwalkers, ndim)*len(Gaps)/2 + len(Gaps)/2
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior_gauss, \
                                    args=[g1,g2,Gaps,p_poor,p_rich,rollover,eps,ndim])
    # "burn-in" period; save final positions and then reset
    pos, prob, state = sampler.run_mcmc(starting_guesses, nburn)
    sampler.reset()
    # sampling period
    sampler.run_mcmc(pos, nsteps)
    #%time sampler.run_mcmc(starting_guesses, nsteps)
    print("done")
    print("Mean acceptance fraction: {0:.3f} (in total {1} steps)"
                    .format(np.mean(sampler.acceptance_fraction),nwalkers*nsteps))

    # sampler.chain is of shape (nwalkers, nsteps, ndim)
    # Let us reshape and all walker chains together
    emcee_trace = sampler.chain[:, :, :].reshape(-1, ndim).T

    transition_pt = search_bins(emcee_trace,Gaps.shape[0])
#    mc_hist1=plt.hist(emcee_trace[0],bins=range(Gaps.shape[0]))
#    mc_hist2=plt.hist(emcee_trace[1],bins=range(Gaps.shape[0]))
#    transition_pt=np.zeros(2)
#    transition_pt[0]=mc_hist1[1][np.where(mc_hist1[0]==max(mc_hist1[0]))][0]+1
#    transition_pt[1]=mc_hist2[1][np.where(mc_hist2[0]==max(mc_hist2[0]))][0]-1
    print("transition points: ",transition_pt)
    return transition_pt, emcee_trace
def mod_dist(x1,x2,mod):
    # finds the shortest difference in modulo
    # useful for determining which tag came first when we expect them to be clumped relative to the rollover time
    if ((x1-x2)%mod)<((x2-x1)%mod):
        return -((x1-x2)%mod)
    else:
        return (x2-x1)%mod
def truncate_poor_region(Data_r,p_poor,p_rich_arr,size,num_blocks,rollover,eps):  # make CondData_r a tuple
    # Uses the transition points for each channel to cut out the dark region
    # determines which channel had first tag using mod_dist, then fills in zeros to rectify array lengths
    # then combines them using Join_columns
    i_array=[]
    new_array=[]
    transition_pt=np.zeros([len(Data_r),2])
    for i in range(len(Data_r)):    
        #TimeData=CondData_r[i][4:,:]
        FullTimeData=Data_r[i][4:,:]
        Gaps=Make_Gaps(FullTimeData)
        transition_pt[i,:] = find_transition(Gaps,p_poor,p_rich_arr[i],size,num_blocks,rollover,eps)[0]
        #del TimeData
        #rollover = rollover * size
        if i==0:
            first_pt=read_time_tag(FullTimeData[:,int(transition_pt[i,0])])
        else:
            if mod_dist(first_pt,read_time_tag(FullTimeData[:,int(transition_pt[i,0])]),rollover)<0:
                first_pt=read_time_tag(FullTimeData[:,int(transition_pt[i,0])])
                #print(first_pt)
        print("first transitions point time tag = ",read_time_tag(FullTimeData[:,int(transition_pt[i,0])]))
        
        i_array.append(ExpandData_sep(FullTimeData[:,int(transition_pt[i,0]):int(transition_pt[i,1])],i))
    for i in range(len(Data_r)):
        FullTimeData=Data_r[i][4:,:]
        print(rollover)
        #print(first_pt)
        #print(read_time_tag(FullTimeData[:,int(transition_pt[i,0])]))
        print(mod_dist(first_pt,read_time_tag(FullTimeData[:,int(transition_pt[i,0])]),rollover))
        new_array.append(np.insert(i_array[i],0,np.zeros(int(mod_dist(first_pt,read_time_tag(FullTimeData[:,int(transition_pt[i,0])]),rollover)))))
        print(int(mod_dist(first_pt,read_time_tag(FullTimeData[:,int(transition_pt[i,0])]),rollover)))
    return Join_columns(new_array)
def sync_n_sift(Ncand,Range,size,num_blocks,mu_rich,mu_dec,d_arr,sort,eta,p_send_arr,\
                method,normalize,sent_name,received_name,length_name):
    # loads in the data specified by Range and sRange, performs the synchronization 
    # uses cruncher to time gate and rectify to Alice's rep rate, then performs sifting, outputting QBER and other stats
    sRange = [Range[0]-Ncand,Range[1]+Ncand]
    #print(sRange)
    Mask = np.ones(Range[1]-Range[0])
    Ntries=0
    recovered_length = np.load(length_name)
    new_sent_data_map = np.memmap(sent_name, dtype='uint8',offset = 9*Range[0], shape=(Range[1]-Range[0],9))
    sent_data = np.array(new_sent_data_map.T)
    del new_sent_data_map
    new_received_data_map = np.memmap(received_name, dtype='uint8',offset = 4*sRange[0], shape=(sRange[1]-sRange[0],4))
    received_data = np.array(new_received_data_map.T)
    print(np.mean(received_data,1))
    del new_received_data_map
    sync_point = sync_pt(Range[0],sent_data,received_data,sRange,recovered_length,\
                     mu_rich,mu_dec,d_arr,sort,eta,size,num_blocks,p_send_arr,method,Ntries,Mask,normalize)
    print(sync_point[0:2])
    del received_data
    data_start = Range[0]
    new_received_data_map = np.memmap(received_name,dtype='uint8',offset = 4*(data_start+sync_point[0]),shape=(Range[1]-Range[0],4))
    received_data = np.array(new_received_data_map.T)
    del new_received_data_map
    test_crunch, crunch_length = cruncher(sent_data,data_start%size,size,num_blocks,sync_point[2])
    stored_crunch = cruncher(received_data,data_start%size,size,num_blocks,sync_point[2])[0]
    print(np.mean(stored_crunch,1))
    del sent_data
    del received_data
    event_arr = np.zeros([9,16])
    #print(crunch_length,test_crunch.shape,stored_crunch.shape)
    for i in range(crunch_length):
        event_arr[np.where(test_crunch[:,i]==1)[0][0],int(read_time_tag(stored_crunch[:,i])[0])]+=1
    SiftedTest, SiftedStored, sift = DataSifter(test_crunch[:4,:],stored_crunch)
    sifted_test_map = np.memmap('sifted_test_mem.npy', dtype='uint8', mode='w+', shape=(SiftedTest.shape[1],4))
    sifted_test_map[:] = SiftedTest.T
    del sifted_test_map
    sifted_stored_map = np.memmap('sifted_stored_mem.npy', dtype='uint8', mode='w+', shape=(SiftedStored.shape[1],4))
    sifted_stored_map[:] = SiftedStored.T
    print(SiftedTest.shape[1],SiftedStored.shape[1])
    del sifted_stored_map
    my_ind = [0,2]
    LR_good = sum(sum(SiftedStored[my_ind[0]:my_ind[1],:]*SiftedTest[my_ind[0]:my_ind[1],:]))
    LR_all = sum(sum(SiftedStored[my_ind[0]:my_ind[1],:]+SiftedTest[my_ind[0]:my_ind[1],:]))/2
    print('L/R accepted = ',LR_all)
    my_ind = [2,4]
    HV_good = sum(sum(SiftedStored[my_ind[0]:my_ind[1],:]*SiftedTest[my_ind[0]:my_ind[1],:]))
    HV_all = sum(sum(SiftedStored[my_ind[0]:my_ind[1],:]+SiftedTest[my_ind[0]:my_ind[1],:]))/2
    print('H/V accepted = ',HV_all) 
    return LR_good,LR_all,HV_good,HV_all,crunch_length,sync_point,event_arr
def cruncher(arr,first,size,num_blocks,whichpoint):
    # time gates num_blocks+1 frames (the +1 determined by "whichpoint" had more signal)
    # rectifies timing to Alice's rep rate, if there are multiple detections for one pulse, these all get rolled into one
    #start = int((first - (size/2-1)+np.floor((num_blocks-1)/2))%size)
    start = (size-first-(1-whichpoint))%size
    leng = int(divmod(arr.shape[1]-start,size)[0])
    first_foot = 0
    last_foot = 0
    new_arr = np.zeros([arr.shape[0],leng])
    not_arr = 1-arr
    for i in range(leng):
        new_arr[:,i] = 1-np.prod(not_arr[:,(i*size+(start)%size):(i*size+(start+num_blocks+1))],1)
    #if start != 0:
    if 0:
        new_arr=np.reshape(np.insert(new_arr.T,0,1-np.prod(not_arr[:,:start],1)),[new_arr.shape[1]+1,arr.shape[0]]).T
        first_foot = 1
    if 0:
    #if ((arr.shape[1] - start)%size) != 0:
        new_arr=np.reshape(np.append(new_arr.T,1-np.prod(not_arr[:,(leng*size+start):],1)),[new_arr.shape[1]+1,arr.shape[0]]).T
        last_foot = 1
    return new_arr, leng+first_foot+last_foot

def LFSR(counts,position):   
    # generates a pseudorandom sequence for Alice to send
    reg = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
    bit = np.zeros(counts)
    for i in range(counts):
        bit[i] = reg[-position]
        feedback = reg[0]^reg[-13]
        reg[:-1] = reg[1:]
        reg[-1] = feedback     
    return bit

def qber_expected(mu_sig,mu_dec,d_arr,sort,eta,p_send_arr,d=2):
    # calculates the expected QBER and accepted sifting fraction for the different inputs using the system characteristics
    p_click_mat_sig = port_prob(mu_sig,d_arr,sort,eta)
    p_click_mat_dec = port_prob(mu_dec,d_arr,sort,eta)
    QBER = np.zeros(2*d)
    fracs = np.zeros(2*d)
    for i in range(2*d):
        half = (i>=d)*d
        p_hit_H = (p_send_arr[i]*p_click_mat_sig[i,i]+p_send_arr[i+2*d]*p_click_mat_dec[i,i])\
                /(p_send_arr[i]+p_send_arr[i+2*d])
        p_hit_V = (p_send_arr[i]*p_click_mat_sig[i,(i+1)%d+half]\
                    +p_send_arr[i+2*d]*p_click_mat_dec[i,(i+1)%d+half])\
                       /(p_send_arr[i]+p_send_arr[i+2*d])
        p_hit_R = (p_send_arr[i]*p_click_mat_sig[i,i-2*half+d]\
                    +p_send_arr[i+2*d]*p_click_mat_dec[i,i-2*half+d])\
                       /(p_send_arr[i]+p_send_arr[i+2*d])
        p_hit_L = (p_send_arr[i]*p_click_mat_sig[i,(i+1)%d-half+d]\
                    +p_send_arr[i+2*d]*p_click_mat_dec[i,(i+1)%d-half+d])\
                       /(p_send_arr[i]+p_send_arr[i+2*d])
        p_wrong = ((1-p_hit_H)*p_hit_V)
        p_right = (p_hit_H*(1-p_hit_V))
        QBER[i] = p_wrong/(p_right+p_wrong)
        fracs[i] = (p_hit_H*(1-p_hit_V)+p_hit_V*(1-p_hit_H))*(1-p_hit_R)*(1-p_hit_L)
    return QBER, fracs

size=8
num_blocks=1
bits=16
rollover=2**bits
poor_darkrate=10e-4  # this should be the measured dark count rate divided by 100 MHz 
# here I had to raise it to get the code to run correctly
#mu_rich = 0.0851648351648352/num_blocks
mu_rich = 0.085/num_blocks  # this is the mean photon number in the signal region

In [ ]:
# generate transmitted data and format

n_hits = 10000000
A = LFSR(n_hits,4)
B = LFSR(n_hits,14)
data = np.zeros([9,n_hits*size])
#data[8,:] = np.ones(n_hits*size)
#L = np.zeros(n_hits*size)
#R = np.zeros(n_hits*size)
#H = np.zeros(n_hits*size)
#V = np.zeros(n_hits*size)
for i in range(n_hits):
    for j in range(num_blocks):
        data[1,i*size+j] = (A[i] == 0 and B[i] == 0)
        data[0,i*size+j] = (A[i] == 0 and B[i] == 1)
        data[2,i*size+j] = (A[i] == 1 and B[i] == 0)
        data[8,i*size+j] = (A[i] == 1 and B[i] == 1)
    #data[8,i*size+num_blocks] = 0
    #data[8,(i+1)*size-1] = 0
#data  = np.array([L,R,H,V,dec,dec,dec,dec,dec])
print('There are ' , np.sum(data[0,:]), 'L counts')
print('There are ' , np.sum(data[1,:]), 'R counts')
print('There are ' , np.sum(data[2,:]), 'H counts')
#print('There are ' , n_hits- H.count(1)-L.count(1)-R.count(1), 'Empty counts')
del A
del B

sent_data_map = np.memmap('lfsr_mem.npy', dtype='uint8', mode='w+', shape=(n_hits*size,9))
sent_data_map[:] = data.T
del data
del sent_data_map

In [ ]:
# This block reads in the measured data

import sys
# import numpy as np
import struct
import os

np.set_printoptions(threshold=sys.maxsize)
# Read file into script
file = open("C:\\Users\\Roddy\\Desktop\\Python QKD\\QKD_receiver_data\\data_table_top_qkd_07_03082020.txt", "rb");
# change to appropriate file path
######data_table_top_qkd_07_03082020
# read from file and make four time arrays
t_0 = np.array([], dtype=np.uint16);
t_1 = np.array([], dtype=np.uint16);
t_2 = np.array([], dtype=np.uint16);
t_3 = np.array([], dtype=np.uint16);

with file as f:
    while True:
        buff = f.read(65538);
        if not buff:
            break
        s = np.frombuffer( buff, dtype=np.dtype('>u2') );
        detector_id = s[16384*2];
        if detector_id == 0:
            t_0 = np.append( t_0, s[:16384*2] )
        elif detector_id == 1:
            t_1 = np.append( t_1, s[:16384*2] )
        elif detector_id == 2:
            t_2 = np.append( t_2, s[:16384*2] )
        elif detector_id == 3:
            t_3 = np.append( t_3, s[:16384*2] )
file.close()
del s
del buff


full_time_tag_tup=[]
full_time_tag_tup.append(make_bin_array(t_0[2:195000],bits))
full_time_tag_tup.append(make_bin_array(t_1[2:192000],bits))
full_time_tag_tup.append(make_bin_array(t_2[2:355000],bits))
full_time_tag_tup.append(make_bin_array(t_3[2:230000],bits))
del t_0
del t_1
del t_2
del t_3

#2:195000
#2:192000
#2:355000
#2:230000
# need to make sure these limits are inside the dark count regions bookending exactly one signal region
# since this data was taken with a repeating 10 million bit pattern being sent

In [ ]:
# This block generates fake qkd data
def superior_fake_data(mu_sig):
    n_hits = 10000000
    padding = 1000000
    A = LFSR(n_hits,4)
    B = LFSR(n_hits,14)
    fake_data = np.zeros([9,n_hits*size+2*padding])
    fake_data[8,:] = np.ones(n_hits*size+2*padding)
    fake_mpn = np.zeros(n_hits*size+2*padding)
    drift = 0
    print(1)
    cnt1=0
    cnt2=0
    cnt3=0
    for i in range(n_hits):
        drift = drift + (np.random.uniform(-1,1)*1*10**(-2))*0
        for j in range(num_blocks):
            #if (i%10000)==0:
            #    print(round(drift))
            #fake_data[8,i*size+j+round(drift)+padding] = 0
            fake_data[1,i*size+j+round(drift)+padding] = (A[i] == 0 and B[i] == 0)
            fake_data[0,i*size+j+round(drift)+padding] = (A[i] == 0 and B[i] == 1)
            fake_data[2,i*size+j+round(drift)+padding] = (A[i] == 1 and B[i] == 0)
            fake_data[8,i*size+j+round(drift)+padding] = (A[i] == 1 and B[i] == 1)
            rel_time = (j+1-drift)%size
            if rel_time>0 and rel_time<1:
                fake_mpn[i*size+j+padding+int(drift//size)*size] += rel_time
                cnt1 +=1
            if rel_time>=1 and rel_time<num_blocks:
                fake_mpn[i*size+j+padding] += 1
                cnt2 +=1
            if rel_time>=num_blocks and rel_time<(num_blocks+1):
                fake_mpn[i*size+j+padding+int((drift+num_blocks)//size)*size] += num_blocks + 1 - rel_time
                cnt3 +=1
    #print(fake_data[:,1500000:1500020])
    print(2)
    print(cnt1)
    print(cnt2)
    print(cnt3)
    print(drift)
    del A
    del B
    #print(np.sum(fake_mpn))
    sent_fakedata_map = np.memmap('fake_data_mem.npy', dtype='uint8', mode='w+', shape=(n_hits*size+2*padding,9))
    sent_fakedata_map[:] = fake_data.T
    del fake_data
    del sent_fakedata_map
    sent_mpn_map = np.memmap('mpn_mem.npy', dtype='float64', mode='w+', shape=(n_hits*size+2*padding))
    sent_mpn_map[:] = fake_mpn.T
    del fake_mpn
    del sent_mpn_map


    #mu_sig = mu_rich
    mu_dec = 0.4 * mu_sig
    sort = np.array([[.5,0,.25,.25],[0,.5,.25,.25],[.25,.25,.5,0],[.25,.25,0,.5]])
    #sort=np.array([[.5,.5,.5,.5],[.5,.5,.5,.5],[.25,.25,.5,0],[.25,.25,.5,0]]) #only basis is known
    eff_simple=np.array([1,1,1,1])
    eta = np.diag(eff_simple)
    #d_arr=np.array([4.8953635620075145e-05,4.256005392075098e-05,5.9906981710614146e-05,6.011140133429999e-05])/size
    d_arr=poor_darkrate*np.ones(4)
    #p_send_arr = num_blocks/(size-2) * np.array([.25,.25,.25,0,0,0,0,0,.25+(size-2)/(num_blocks*(size-2-num_blocks))])
    p_send_arr = [.25,.25,.25,0,0,0,0,0,.25]


    Range = [0,n_hits*size+2*padding]
    Stored = np.zeros((4,Range[1]-Range[0]))
    opN=1000000
    nn,rem = divmod(Range[1],opN)
    #print(nn,rem)
    for i in range(nn):
        new_fake_data_map = np.memmap('fake_data_mem.npy', dtype='uint8',mode = 'r',offset = (9*i*opN), shape=(opN,9))
        fake_data = np.array(new_fake_data_map.T)
        new_mpn_map = np.memmap('mpn_mem.npy', dtype='float64',mode = 'r',offset = (8*i*opN), shape=(opN))
        mpn_arr = np.array(new_mpn_map.T)
        del new_fake_data_map
        del new_mpn_map

        Stored[:,(i*opN):((i+1)*opN)] = FakeData_R(fake_data,mpn_arr,mu_sig,mu_dec,d_arr,sort,eta)
        #print(mpn_arr[0])
        del fake_data
        del mpn_arr
    if rem!=0:
        new_fake_data_map = np.memmap('fake_data_mem.npy', dtype='uint8',mode = 'r',offset = 9*nn*opN, shape=(rem,9))
        fake_data = np.array(new_fake_data_map.T)
        new_mpn_map = np.memmap('mpn_mem.npy', dtype='float64',mode = 'r',offset = 8*nn*opN, shape=(rem))
        mpn_arr = np.array(new_mpn_map.T)
        del new_fake_data_map
        del new_mpn_map

        Stored[:,(nn*opN):] = FakeData_R(fake_data,mpn_arr,mu_sig,mu_dec,d_arr,sort,eta)
        del fake_data
        del mpn_arr
    # this array lets us skip packing and unpacking into the same format as the FPGA output
    stored_data_map = np.memmap('stored_data_mem.npy', dtype='uint8',mode = 'w+', shape=(n_hits*size,4))
    stored_data_map[:] = Stored[:,padding:-padding].T
    del stored_data_map
    np.save('stored_length',n_hits*size)
    t0 = []
    t1 = []
    t2 = []
    t3 = []
    #tup = [t0,t1,t2,t3]
    for i in range(Stored.shape[1]):
        if Stored[0,i] == 1:
            t0.append(i%(2**bits))
        if Stored[1,i] == 1:
            t1.append(i%(2**bits))
        if Stored[2,i] == 1:
            t2.append(i%(2**bits))
        if Stored[3,i] == 1:
            t3.append(i%(2**bits))
    t0 = np.array(t0,dtype=np.uint64)
    t1 = np.array(t1,dtype=np.uint64)
    t2 = np.array(t2,dtype=np.uint64)
    t3 = np.array(t3,dtype=np.uint64)
    fake_time_tag_tup = []
    fake_time_tag_tup.append(make_bin_array(t0,bits))
    fake_time_tag_tup.append(make_bin_array(t1,bits))
    fake_time_tag_tup.append(make_bin_array(t2,bits))
    fake_time_tag_tup.append(make_bin_array(t3,bits))
    del t0
    del t1
    del t2
    del t3
    del Stored
    return 1

#print(Stored)
#received_fakedata_map = np.memmap('received_fake_data_mem.npy', dtype='uint8', mode='w+', shape=(n_hits*size+2*padding,4))
#received_fakedata_map[:] = Stored.T


#del received_fakedata_map


In [ ]:
# This block identifies the transition points at the edges of the signal region, and slices the signal data
full_time_tag_tup = fake_time_tag_tup  #use fake data
#p_poor=1-(1-poor_darkrate)**size
p_poor = poor_darkrate
#p_rich=1-(.25*np.exp(-mu_rich/(2*num_blocks))+.5*np.exp(-mu_rich/(4*num_blocks))+.25)
p_click_mat_sig = port_prob(mu_rich,d_arr,sort,eta)
p_click_mat_dec = port_prob(.4*mu_rich,d_arr,sort,eta)
p_click_mat = np.zeros([len(p_send_arr),sort.shape[0]])
p_click_mat[:sort.shape[0],:] = p_click_mat_sig[:sort.shape[0],:]
p_click_mat[sort.shape[0]:,:] = p_click_mat_dec
p_rich_arr = nlik_prob(p_click_mat,p_send_arr)
#p_rich=1-(.25*np.exp(-mu_rich/(2*num_blocks))+.5*np.exp(-mu_rich/(4*num_blocks))+.25)
#p_rich=1-(1-poor_darkrate)**size*(.25*np.exp(-mu_rich/2)+.5*np.exp(-mu_rich/4)+.25)
#p_rich=1-(1-poor_darkrate)**size*(.25*np.exp(-mu_rich)+.75)

# R,L,H,V
eps=10e-4
recovered_data=truncate_poor_region(full_time_tag_tup,p_poor,p_rich_arr,size,num_blocks,rollover,eps)
recovered_length = recovered_data.shape[1]
del full_time_tag_tup
received_data_map = np.memmap('received_mem.npy', dtype='uint8', mode='w+', shape=(recovered_data.T.shape))
received_data_map[:] = recovered_data.T
del recovered_data
del received_data_map
# Mean acceptance fractions should be higher than .4 I think.  
#  The real proof is whether the transition point time tags are close to each other (usually within a few hundred)
print(recovered_length)
np.save('recovered_length',recovered_length)

In [ ]:
#%%time 
# This block performs synchronization in batches and reads out QBERs, etc.
#sort = np.array([[.5,0,.25,.25],[0,.5,.25,.25],[.25,.25,.5,0],[.25,.25,0,.5]])
sort=np.array([[.25,.25,.25,.25],[.25,.25,.25,.25],[.25,.25,.5,0],[.25,.25,.5,0]]) #only basis is known
eff_simple=np.array([1,1,1,1])
eta = np.diag(eff_simple)
#d_arr=np.array([4.8953635620075145e-05,4.256005392075098e-05,5.9906981710614146e-05,6.011140133429999e-05])/size
d_arr=poor_darkrate*np.ones(4)
p_send_arr = [.25,.25,.25,0,0,0,0,0,.25]
#mu_rich = 0.0851648351648352/num_blocks
mu_dec = 0.4 * mu_rich
#poor_darkrate=(5)*4.983147923908952e-05
#filenames=['lfsr_mem.npy','stored_data_mem.npy','stored_length.npy'] # use simulated data directly
filenames=['lfsr_mem.npy','received_mem.npy','recovered_length.npy']
method = 'fft'
normalize=1
start = 1000000
nn=500
batch = 100000
Ncand = 2000
QBs = np.zeros([5,nn])
rolling_events = np.zeros((9,16))
sync_arr = np.zeros(500)
for i in range(nn):
    Range = ([int(start+batch*i),int(start+batch*(i+1))])
    LR_good,LR_all,HV_good,HV_all,crunch_length,sync_point,event_arr \
    = sync_n_sift(Ncand,Range,size,num_blocks,mu_rich/2,mu_dec,d_arr,sort,eta,p_send_arr,method,normalize,\
                  filenames[0],filenames[1],filenames[2])
    QBs[:,i] = [LR_good,LR_all,HV_good,HV_all,crunch_length]
    print(' ')
    rolling_events=rolling_events+event_arr
    sync_arr[i] = sync_point[0]

QBs_tot = np.sum(QBs,1)
print('Accepted fraction = ',(QBs_tot[1]+QBs_tot[3])/QBs_tot[4])
print('L/R QBER = ',1-QBs_tot[0]/QBs_tot[1])
print('H/V QBER = ',1-QBs_tot[2]/QBs_tot[3])

#print(rolling_events)

#outcomes_arr = np.zeros([12,6])
#cnt = 0
#my_metric_map = np.memmap('metric_data_mem.npy', dtype='float64',offset = 0, shape=(4000))
#my_metric = np.array(my_metric_map.T)
#del my_metric_map

#metric_data_map = np.memmap('metric_data_'+str(batch)+'.npy', dtype='float64',mode = 'w+', shape=(my_metric.shape[0]))
#metric_data_map[:] = my_metric.T
#del metric_data_map

drift_data_map = np.memmap('real_drift_data.npy', dtype='float64',mode = 'w+', shape=(500))
drift_data_map[:] = sync_arr.T
del drift_data_map

In [ ]:
# show the clock drift over time as measured by the synchronization algorithm
my_drift_map = np.memmap('real_drift_data.npy', dtype='float64',offset = 0, shape=(500))
sync_arr = np.array(my_drift_map.T)
del my_drift_map

fig_true = plt.figure(figsize=(8,12))
ax_true = fig_true.add_subplot(2,1,1)
ax_true.plot(np.arange(sync_arr.shape[0])*100000*10*10**(-9),sync_arr*1e-2)
print((sync_arr[-1]-sync_arr[0])*10/(sync_arr.shape[0]*100000*10))
plt.ylabel('relative clock offset ($\mu s$)')
plt.xlabel('lab time (s)')

plt.savefig('qkd_clock_drift_fig.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
#%%time
# This block compares the probabilities to reality
# WARNING: 3 hour run time
#sort = np.array([[.5,0,.25,.25],[0,.5,.25,.25],[.25,.25,.5,0],[.25,.25,0,.5]])
sort=np.array([[.25,.25,.25,.25],[.25,.25,.25,.25],[.25,.25,.5,0],[.25,.25,.5,0]]) #only basis is known
eff_simple=np.array([1,1,1,1])
eta = np.diag(eff_simple)
#d_arr=np.array([4.8953635620075145e-05,4.256005392075098e-05,5.9906981710614146e-05,6.011140133429999e-05])/size
d_arr=poor_darkrate*np.ones(4)
p_send_arr = [.25,.25,.25,0,0,0,0,0,.25]

mu_rich = 0.05
mu_dec = 0.4 * mu_rich # these simulations did not include intermediate decoys but they could
#poor_darkrate=(5)*4.983147923908952e-05
superior_fake_data(mu_rich)
filenames=['lfsr_mem.npy','stored_data_mem.npy','stored_length.npy']
#filenames=['lfsr_mem.npy','received_mem.npy','recovered_length.npy']
method = 'fft'
normalize=1
start = 20000
nn=1
batch = 3200000
Ncand = 10000
QBs = np.zeros([5,nn])
for i in range(nn):
    Range = ([int(start+batch*i),int(start+batch*(i+1))])
    LR_good,LR_all,HV_good,HV_all,crunch_length,sync_point,event_arr \
    = sync_n_sift(Ncand,Range,size,num_blocks,mu_rich,mu_dec,d_arr,sort,eta,p_send_arr,method,normalize,\
                  filenames[0],filenames[1],filenames[2])
    
ref_sync = sync_point[0]
batch = 20
N=1000
outcomes_arr = np.zeros([150,6])
cnt = 0
while (batch) < (8*8000000/N):
    
    Ncand = 2000
    probs = np.zeros(N)
    points = np.zeros(N)
    for i in range(N):
        Range = ([int(start+batch*i),int(start+batch*(i+1))])
        LR_good,LR_all,HV_good,HV_all,crunch_length,sync_point,event_arr \
        = sync_n_sift(Ncand,Range,size,num_blocks,mu_rich,mu_dec,d_arr,sort,eta,p_send_arr,method,normalize,\
                      filenames[0],filenames[1],filenames[2])
        points[i] = sync_point[0]
        probs[i] = sync_point[1]
    success_rate = np.sum((points==ref_sync))/N
    projected_rate = np.sum(probs)/N
    outcomes_arr[cnt,0] = success_rate
    outcomes_arr[cnt,1] = projected_rate
    outcomes_arr[cnt,2] = np.sqrt(success_rate*(1-success_rate)/N)
    outcomes_arr[cnt,3] = np.std(probs)/np.sqrt(N)      #np.sqrt(projected_rate*(1-projected_rate)/N)
    outcomes_arr[cnt,4] = batch
    outcomes_arr[cnt,5] = N
    cnt = cnt+1
    batch = int(1.5*batch) 
file_name = 'outcomes_' + str(mu_rich) +'_6_20_21_mem.npy'
#print(file_name)
outcomes_map = np.memmap(file_name, dtype='float64', mode='w+', shape=(6,cnt))
outcomes_map[:] = outcomes_arr[:cnt,:].T
del outcomes_map

In [ ]:
# Plots probabilities vs. reality as simulated in the previous block
cnt=20
mu_label = '0.05'
file_name = 'outcomes_' + mu_label +'_6_10_21_mem.npy'
new_outcomes_map = np.memmap(file_name, dtype='float64',offset = 0, shape=(6,cnt))
outcomes_arr = np.array(new_outcomes_map.T)
del new_outcomes_map

fig_true = plt.figure(figsize=(8,12))
ax_true = fig_true.add_subplot(2,1,1)
ax_true.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(outcomes_arr[:cnt,0]),\
                 yerr=(outcomes_arr[:cnt,2])/((outcomes_arr[:cnt,0])*np.log(10)),ls='-',label='log$_{10}$ f(S$_j$|B$_1$,...,B$_{M+N}$)')
ax_true.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(outcomes_arr[:cnt,1]),\
                 yerr=(outcomes_arr[:cnt,3])/((outcomes_arr[:cnt,1])*np.log(10)),ls='-.',label='log$_{10}$ p(S$_j$|B$_1$,...,B$_{M+N}$)')

ax_true.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(1-outcomes_arr[:cnt,0]),\
                 yerr=(outcomes_arr[:cnt,2])/((1-outcomes_arr[:cnt,0])*np.log(10)),ls='--',label='log$_{10}$(1-f(S$_j$|B$_1$,...,B$_{M+N}$))')
ax_true.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(1-outcomes_arr[:cnt,1]),\
                 yerr=(outcomes_arr[:cnt,3])/((1-outcomes_arr[:cnt,1])*np.log(10)),ls=':',label='log$_{10}$(1-p(S$_j$|B$_1$,...,B$_{M+N}$))')
#plt.ylabel('log$_{10}$ p(S$_j$|B$_1$,...,B$_{M+N}$)')
plt.xlabel('log$_{10}$N')
plt.title('$\mu = $' + mu_label)


mu_label = '1.0'
file_name = 'outcomes_' + mu_label +'_6_10_21_mem.npy'
new_outcomes_map = np.memmap(file_name, dtype='float64',offset = 0, shape=(6,cnt))
outcomes_arr = np.array(new_outcomes_map.T)
del new_outcomes_map
ax_true2 = fig_true.add_subplot(2,1,2)
ax_true2.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(outcomes_arr[:cnt,0]),\
                 yerr=(outcomes_arr[:cnt,2])/((outcomes_arr[:cnt,0])*np.log(10)),ls='-',label='log$_{10}$ f(S$_j$|B$_1$,...,B$_{M+N}$)')
ax_true2.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(outcomes_arr[:cnt,1]),\
                 yerr=(outcomes_arr[:cnt,3])/((outcomes_arr[:cnt,1])*np.log(10)),ls='-.',label='log$_{10}$ p(S$_j$|B$_1$,...,B$_{M+N}$)')

ax_true2.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(1-outcomes_arr[:cnt,0]),\
                 yerr=(outcomes_arr[:cnt,2])/((1-outcomes_arr[:cnt,0])*np.log(10)),ls='--',label='log$_{10}$(1-f(S$_j$|B$_1$,...,B$_{M+N}$))')
ax_true2.errorbar(np.log10(outcomes_arr[:cnt,4]),np.log10(1-outcomes_arr[:cnt,1]),\
                 yerr=(outcomes_arr[:cnt,3])/((1-outcomes_arr[:cnt,1])*np.log(10)),ls=':',label='log$_{10}$(1-p(S$_j$|B$_1$,...,B$_{M+N}$))')

#plt.ylabel('log$_{10}$ p(S$_j$|B$_1$,...,B$_{M+N}$)')
plt.xlabel('log$_{10}$N')
plt.title('$\mu = $' + mu_label)
ax_true.legend()
ax_true2.legend(loc=4)
ax_true.text(.5,0,'a)')
ax_true2.text(.5,0,'b)')
ax_true.set_xlim(1.2,6)
ax_true2.set_xlim(1.2,6)
plt.subplots_adjust(hspace=0.3)
print(cnt)

plt.savefig('probability_test.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
# Plots sync confidence vs. mu using simulated data generated above
cnt = 20
name_list = ['outcomes_1.0_6_10_21_mem.npy','outcomes_0.5_6_10_21_mem.npy','outcomes_0.1_6_10_21_mem.npy',\
             'outcomes_0.05_6_10_21_mem.npy','outcomes_0.01_6_10_21_mem.npy','outcomes_0.005_6_10_21_mem.npy']
style = ['solid',(0, (1, 10)),'dashed','dashdot',(0, (1, 1)),(0, (3, 5, 1, 5, 1, 5))]
fig_true = plt.figure(figsize=(12,16))
ax_true = fig_true.add_subplot(2,1,1)
for i in range(len(name_list)):
    
    file_name = name_list[i]
    new_outcomes_map = np.memmap(file_name, dtype='float64',offset = 0, shape=(6,cnt))
    outcomes_arr = np.array(new_outcomes_map.T)
    del new_outcomes_map
    
    ax_true.plot(np.log10(outcomes_arr[:cnt,4]),np.log10(outcomes_arr[:cnt,1]),\
                 ls=style[i],label='$\mu = $'+file_name[9:-16])
plt.ylabel('log$_{10}$ p(S$_j$|B$_1$,...,B$_{M+N}$)')
plt.xlabel('log$_{10}$N')
plt.title('Average Confidence')
#plt.title('$\mu = $' + mu_label)
ax_true.legend()
ax_true.set_xlim(1.2,5)
plt.savefig('sync_confidence_parameterized.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1,
        frameon=None, metadata=None)

In [ ]:
# This block finds string length necessary to achieve 95% synchronization confidence
# WARNING: multi-hour run time
sort=np.array([[.25,.25,.25,.25],[.25,.25,.25,.25],[.25,.25,.5,0],[.25,.25,.5,0]]) #only basis is known
eff_simple=np.array([1,1,1,1])
eta = np.diag(eff_simple)
#d_arr=np.array([4.8953635620075145e-05,4.256005392075098e-05,5.9906981710614146e-05,6.011140133429999e-05])/size
d_arr=poor_darkrate*np.ones(4)

p_send_arr = [.25,.25,.25,0,0,0,0,0,.25]
method = 'fft'
normalize=1
filenames=['lfsr_mem.npy','stored_data_mem.npy','stored_length.npy']
ref_sync = 0
N=100
start = 20000
Ncand = 2000
def f_outcomes(batch,mu):
    outcomes_arr = np.zeros([6,1])
    probs = np.zeros(N)
    points = np.zeros(N)
    for i in range(N):
        Range = ([int(start+batch*i),int(start+batch*(i+1))])
        LR_good,LR_all,HV_good,HV_all,crunch_length,sync_point,event_arr \
        = sync_n_sift(Ncand,Range,size,num_blocks,mu,.4*mu,d_arr,sort,eta,p_send_arr,method,normalize,\
                        filenames[0],filenames[1],filenames[2])
        points[i] = sync_point[0]
        probs[i] = sync_point[1]
    success_rate = np.sum((points==ref_sync))/N
    projected_rate = np.sum(probs)/N
    outcomes_arr[0] = success_rate
    outcomes_arr[1] = projected_rate
    outcomes_arr[2] = np.sqrt(success_rate*(1-success_rate)/N)
    outcomes_arr[3] = np.std(probs)/np.sqrt(N)      #np.sqrt(projected_rate*(1-projected_rate)/N)
    outcomes_arr[4] = batch
    outcomes_arr[5] = N
    return outcomes_arr

mus=np.logspace(-2,0,25)
outcomes_perm = np.zeros([25,7])
for j in range(25):
    superior_fake_data(mus[j])
    cnt = 0
    #batch = 20
    projected_rate = 0
    f_a = f_outcomes(100,mus[j])
    f_b = f_outcomes(int(8*8000000/N),mus[j])
    a = 100
    b = int(8*8000000/N)
    stop = 0
    while (stop==0):
        if f_b[1]<.95:
            outcomes_arr = f_b
            stop = 1
        
        c = int((a+b)/2)        
        if ((c == a) or (c == b)):
            outcomes_arr = f_b
            stop = 1
        f_c = f_outcomes(c,mus[j])
        if f_c[1]<.95:
            a = c
            f_a = f_c
        else:
            b = c
            f_b = f_c
        
        cnt = cnt+1
        #batch = int(1.5*batch) 
    outcomes_perm[j,:-1] = outcomes_arr[:,0]
    outcomes_perm[j,-1] = mus[j]
file_name = 'synclength_vs_confidence_5_08_21_mem.npy'
#print(file_name)
outcomes_map = np.memmap(file_name, dtype='float64', mode='w+', shape=(7,25))
outcomes_map[:] = outcomes_perm.T
del outcomes_map

In [ ]:
# Plots string length necessary to achieve 95% synchronization confidence vs. mu as simulated in previous block
new_outcomes_map = np.memmap('synclength_vs_confidence_4_30_21_mem.npy', dtype='float64',offset = 0, shape=(7,25))
outcomes_perm = np.array(new_outcomes_map.T)
del new_outcomes_map

fig_true = plt.figure(figsize=(8,12))
ax_true = fig_true.add_subplot(2,1,1)
ax_true.plot(np.log10(outcomes_perm[:,6]),np.log10(outcomes_perm[:,4]),'-',label=r'd = 8 $\times$ 10$^{-4}$')
plt.ylabel('log$_{10}$N')
plt.xlabel('log$_{10}(\mu)$')
plt.title('Threshold for 95% confidence')
print(outcomes_perm[:,4])

new_outcomes_map = np.memmap('synclength_vs_confidence_5_08_21_mem.npy', dtype='float64',offset = 0, shape=(7,25))
outcomes_perm = np.array(new_outcomes_map.T)
del new_outcomes_map
ax_true.plot(np.log10(outcomes_perm[:,6]),np.log10(outcomes_perm[:,4]),'--',label=r'd = 8 $\times$ 10$^{-3}$')

new_outcomes_map = np.memmap('synclength_vs_confidence_5_06_21_mem.npy', dtype='float64',offset = 0, shape=(7,25))
outcomes_perm = np.array(new_outcomes_map.T)
del new_outcomes_map
ax_true.plot(np.log10(outcomes_perm[:,6]),np.log10(outcomes_perm[:,4]),':',label=r'd = 8 $\times$ 10$^{-2}$')

ax_true.legend()

plt.savefig('sync_confidence_95.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.1,
        frameon=None, metadata=None)